In [1]:
!pip install -q seaborn

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # check device

In [ ]:
import pathlib  
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint

print(tf.__version__) # check tensorflow version

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/Dataset_example_0.600_0.601sec_6input.xlsx'

In [ ]:
pre_dataset = pd.read_excel(filename)
pre_dataset = pre_dataset.sample(frac=1).reset_index(drop=True)
pre_dataset.tail()

In [ ]:
# statistics for training dataset
train_stats = pre_dataset.describe()
train_stats.pop("Temperature")
train_stats = train_stats.transpose()
train_stats

In [ ]:
# labeling for training dataset
dataset = pre_dataset
train_labels = dataset.pop('Temperature')

In [ ]:
# normalization for training datset
def norm(x):
    return (x-train_stats['mean'])/train_stats['std']
normed_train_data = norm(dataset)

In [ ]:
print(normed_train_data)
print(train_labels)

In [ ]:
# neural network model
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(dataset.keys())]),
        layers.Dense(64, activation='relu'),
        #layers.Dense(64, activation='relu'),
        #layers.Dense(64, activation='relu'),
        #layers.Dense(64, activation='relu'),
        layers.Dense(1)    
        
    ])
    optimizer = tf.keras.optimizers.Adam(0.001) 
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
 
    return model

In [ ]:
# build model
model = build_model()
model.summary()

In [ ]:
# checkpoint_path
import os

checkpoint_path = "training_trash/cp_Temp.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                   save_weights_only=True, save_best_only=True, monitor = 'val_mse',
                                   verbose=1)# Train the model with 

In [ ]:
# machine training
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 50

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[cp_callback])

In [ ]:
# check val_loss with epoch
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist

In [ ]:
# plot mse (training and validation)
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure(figsize=(8,12))

  plt.subplot(2,1,1)
  plt.xlabel('Epoch')
  plt.ylabel('Mean square error')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label='Validation Error')
  plt.ylim(0,0.2)
  plt.legend()
  plt.show()
    
plot_history(history)

In [ ]:
# prediction for training datset
model.load_weights("training_trash/cp_Temp.ckpt")

# train data results plot
train_predictions = model.predict(normed_train_data).flatten()
train_predictions
plt.scatter(train_labels, train_predictions)
plt.xlabel('True Values [T]')
plt.ylabel('Predictions [T]')
plt.axis('equal')
plt.axis('square')
plt.xlim([-20,20]) #plt.xlim()[1]])
plt.ylim([-20,20]) #plt.ylim()[1]])
_ = plt.plot([-100, 500], [-100, 500])